In [1]:
from langchain.llms import GooglePalm

In [2]:
import os
import streamlit as st
import pickle
import time
import langchain
from langchain import OpenAI
from langchain.chains import RetrievalQAWithSourcesChain
from langchain.chains.qa_with_sources.loading import load_qa_with_sources_chain
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.document_loaders import UnstructuredURLLoader
from langchain.embeddings import OpenAIEmbeddings
from langchain.vectorstores import FAISS
import requests
from bs4 import BeautifulSoup

In [23]:
!pip3 install unstructured libmagic python-magic python-magic-bin


[notice] A new release of pip is available: 24.1.2 -> 24.3.1
[notice] To update, run: python.exe -m pip install --upgrade pip


In [24]:
from langchain.document_loaders import UnstructuredURLLoader

In [32]:
new_urls=["https://timesofindia.indiatimes.com/city/kochi/centre-calls-health-meet-as-covid-rises-in-kerala-10-deaths-this-month/articleshow/106106851.cms"]


In [33]:
loader=UnstructuredURLLoader(urls=new_urls)

In [34]:
data=loader.load()

2024-12-18 16:29:22.705 INFO    unstructured: Reading document from string ...
2024-12-18 16:29:22.717 INFO    unstructured: Reading document ...


In [35]:
data[0]

Document(page_content="Edition\n\nIN\n\nIN\n\nUS\n\nSign In\n\nTOI\n\nCity\n\nkochi\n\nmumbai\n\ndelhi\n\nbengaluru\n\nHyderabad\n\nkolkata\n\nchennai\n\nagra\n\nagartala\n\nahmedabad\n\najmer\n\namaravati\n\namritsar\n\nbareilly\n\nbhubaneswar\n\nbhopal\n\nchandigarh\n\nchhatrapati sambhajinagar\n\ncoimbatore\n\ncuttack\n\ndehradun\n\nerode\n\nfaridabad\n\nghaziabad\n\ngoa\n\ngurgaon\n\nguwahati\n\nhubballi\n\nimphal\n\nindore\n\nitanagar\n\njaipur\n\njammu\n\njamshedpur\n\njodhpur\n\nkanpur\n\nkohima\n\nkolhapur\n\nkozhikode\n\nludhiana\n\nlucknow\n\nmadurai\n\nmangaluru\n\nmeerut\n\nmumbai region\n\nmysuru\n\nnagpur\n\nnashik\n\nnavi mumbai\n\nnoida\n\npatna\n\nprayagraj\n\npuducherry\n\npune\n\nraipur\n\nrajkot\n\nranchi\n\nthane\n\nsalem\n\nshillong\n\nshimla\n\nsrinagar\n\nsurat\n\ntrichy\n\nthiruvananthapuram\n\nudaipur\n\nvadodara\n\nvaranasi\n\nvijayawada\n\nvisakhapatnam\n\nphotos\n\nWeb Stories\n\nToday's ePaper\n\nkerala elections\n\nphotos\n\nweather\n\nNews\n\nCity News\n

In [40]:
text_splitter =RecursiveCharacterTextSplitter(
    separators=["\n\n\n\n","\n\n\n","\n\n", "\n" ],
    chunk_size=1000,
    chunk_overlap=200
)

In [41]:
docs=text_splitter.split_documents(data)
len(docs)

15

In [42]:
docs

[Document(page_content="Edition\n\nIN\n\nIN\n\nUS\n\nSign In\n\nTOI\n\nCity\n\nkochi\n\nmumbai\n\ndelhi\n\nbengaluru\n\nHyderabad\n\nkolkata\n\nchennai\n\nagra\n\nagartala\n\nahmedabad\n\najmer\n\namaravati\n\namritsar\n\nbareilly\n\nbhubaneswar\n\nbhopal\n\nchandigarh\n\nchhatrapati sambhajinagar\n\ncoimbatore\n\ncuttack\n\ndehradun\n\nerode\n\nfaridabad\n\nghaziabad\n\ngoa\n\ngurgaon\n\nguwahati\n\nhubballi\n\nimphal\n\nindore\n\nitanagar\n\njaipur\n\njammu\n\njamshedpur\n\njodhpur\n\nkanpur\n\nkohima\n\nkolhapur\n\nkozhikode\n\nludhiana\n\nlucknow\n\nmadurai\n\nmangaluru\n\nmeerut\n\nmumbai region\n\nmysuru\n\nnagpur\n\nnashik\n\nnavi mumbai\n\nnoida\n\npatna\n\nprayagraj\n\npuducherry\n\npune\n\nraipur\n\nrajkot\n\nranchi\n\nthane\n\nsalem\n\nshillong\n\nshimla\n\nsrinagar\n\nsurat\n\ntrichy\n\nthiruvananthapuram\n\nudaipur\n\nvadodara\n\nvaranasi\n\nvijayawada\n\nvisakhapatnam\n\nphotos\n\nWeb Stories\n\nToday's ePaper\n\nkerala elections\n\nphotos\n\nweather\n\nNews\n\nCity News\

In [44]:
with open("main.txt", "r") as f:
    api_key=f.readline().strip()

In [46]:
!pip install sentence-transformers faiss-cpu



[notice] A new release of pip is available: 24.1.2 -> 24.3.1
[notice] To update, run: python.exe -m pip install --upgrade pip


In [47]:
from sentence_transformers import SentenceTransformer
from langchain.vectorstores import FAISS
from langchain.docstore.document import Document

# Load a SentenceTransformers model
model = SentenceTransformer('all-MiniLM-L6-v2')

# Define a custom embeddings function
class CustomEmbeddings:
    def embed_documents(self, texts):
        """Embed a list of texts (for documents)."""
        return model.encode(texts, convert_to_tensor=True).tolist()

    def embed_query(self, text):
        """Embed a single query text."""
        return model.encode(text, convert_to_tensor=True).tolist()


embeddings = CustomEmbeddings()

# Build FAISS vector store
vector_index = FAISS.from_documents(docs, embeddings)

2024-12-18 17:07:48.155 INFO    sentence_transformers.SentenceTransformer: Load pretrained SentenceTransformer: all-MiniLM-L6-v2


2024-12-18 17:09:24.328 INFO    sentence_transformers.SentenceTransformer: Use pytorch device: cpu


Batches:   0%|          | 0/1 [00:00<?, ?it/s]

2024-12-18 17:09:30.360 INFO    faiss.loader: Loading faiss with AVX2 support.
2024-12-18 17:09:30.363 INFO    faiss.loader: Could not load library with AVX2 support due to:
ModuleNotFoundError("No module named 'faiss.swigfaiss_avx2'")
2024-12-18 17:09:30.365 INFO    faiss.loader: Loading faiss.
2024-12-18 17:09:31.058 INFO    faiss.loader: Successfully loaded faiss.


In [48]:
import pickle

with open("vector_index.pkl","wb") as file:
    pickle.dump(vector_index, file)

In [50]:
import os 
if os.path.exists("vector_index.pkl"):
    with open("vector_index.pkl","rb") as fb:
        vector_index_transformer=pickle.load(fb)

In [56]:
from langchain.chat_models import ChatOpenAI

In [57]:
llm = ChatOpenAI(
    model="gpt-3.5-turbo", 
    openai_api_key=api_key
)

In [58]:
#llm=OpenAI(temperature=0.9, max_tokens=500, openai_api_key=api_key)

In [59]:
chain=RetrievalQAWithSourcesChain.from_llm(llm=llm, retriever=vector_index_transformer.as_retriever())
chain

RetrievalQAWithSourcesChain(memory=None, callbacks=None, callback_manager=None, verbose=False, tags=None, metadata=None, combine_documents_chain=MapReduceDocumentsChain(memory=None, callbacks=None, callback_manager=None, verbose=False, tags=None, metadata=None, input_key='input_documents', output_key='output_text', llm_chain=LLMChain(memory=None, callbacks=None, callback_manager=None, verbose=False, tags=None, metadata=None, prompt=PromptTemplate(input_variables=['context', 'question'], output_parser=None, partial_variables={}, template='Use the following portion of a long document to see if any of the text is relevant to answer the question. \nReturn any relevant text verbatim.\n{context}\nQuestion: {question}\nRelevant text, if any:', template_format='f-string', validate_template=True), llm=ChatOpenAI(cache=None, verbose=False, callbacks=None, callback_manager=None, tags=None, metadata=None, client=<class 'openai.api_resources.chat_completion.ChatCompletion'>, model_name='gpt-3.5-tur

In [60]:
query="What is the number of covid case reported on 16 December?"

In [61]:
langchain.debug=True

In [62]:
chain({"question":query}, return_only_outputs=True)

[chain/start] [1:chain:RetrievalQAWithSourcesChain] Entering Chain run with input:
{
  "question": "What is the number of covid case reported on 16 December?"
}


Batches:   0%|          | 0/1 [00:00<?, ?it/s]

[chain/start] [1:chain:RetrievalQAWithSourcesChain > 3:chain:MapReduceDocumentsChain] Entering Chain run with input:
[inputs]
[chain/start] [1:chain:RetrievalQAWithSourcesChain > 3:chain:MapReduceDocumentsChain > 4:chain:LLMChain] Entering Chain run with input:
{
  "input_list": [
    {
      "context": "Durgesh Nandan Jha / TNN /\n\nUpdated: Dec 19, 2023, 09:33 IST\n\nShare\n\nAA\n\nSmall\n\nMedium\n\nLarge\n\nFollow us\n\nImage used for representational purpose only\n\nKerala is witnessing a rise in Covid cases as well as deaths. Daily cases detected in the state have nearly tripled in a week while 10 deaths have been reported from December 1 to 17, as per data sent to the Centre, sources in the state health department said.\n\nOf particular concern is JN.1, a new Covid sub-variant recently detected in a 79-year-old woman in Kerala.\n\nOn December 16, Kerala reported 302 new Covid cases and four deaths, the highest in the latest outbreak. On December 10, Kerala had reported 109 cases

2024-12-18 17:36:42.359 INFO    openai: error_code=insufficient_quota error_message='You exceeded your current quota, please check your plan and billing details. For more information on this error, read the docs: https://platform.openai.com/docs/guides/error-codes/api-errors.' error_param=None error_type=insufficient_quota message='OpenAI API error received' stream_error=False
2024-12-18 17:36:42.363 WARNING langchain.llms.base: Retrying langchain.chat_models.openai.ChatOpenAI.completion_with_retry.<locals>._completion_with_retry in 4.0 seconds as it raised RateLimitError: You exceeded your current quota, please check your plan and billing details. For more information on this error, read the docs: https://platform.openai.com/docs/guides/error-codes/api-errors..
2024-12-18 17:36:46.707 INFO    openai: error_code=insufficient_quota error_message='You exceeded your current quota, please check your plan and billing details. For more information on this error, read the docs: https://platfo

[chain/error] [1:chain:RetrievalQAWithSourcesChain > 3:chain:MapReduceDocumentsChain > 4:chain:LLMChain > 5:llm:ChatOpenAI] [32.42s] Llm run errored with error:
"RateLimitError(message='You exceeded your current quota, please check your plan and billing details. For more information on this error, read the docs: https://platform.openai.com/docs/guides/error-codes/api-errors.', http_status=429, request_id=None)"
[chain/error] [1:chain:RetrievalQAWithSourcesChain > 3:chain:MapReduceDocumentsChain > 4:chain:LLMChain] [32.43s] Chain run errored with error:
"RateLimitError(message='You exceeded your current quota, please check your plan and billing details. For more information on this error, read the docs: https://platform.openai.com/docs/guides/error-codes/api-errors.', http_status=429, request_id=None)"
[chain/error] [1:chain:RetrievalQAWithSourcesChain > 3:chain:MapReduceDocumentsChain] [32.43s] Chain run errored with error:
"RateLimitError(message='You exceeded your current quota, plea

RateLimitError: You exceeded your current quota, please check your plan and billing details. For more information on this error, read the docs: https://platform.openai.com/docs/guides/error-codes/api-errors.